In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import datetime
from time import time
from tqdm import tqdm
import torch

import src.mace.utils as utils
import src.mace.input as input
import src.mace.dataset as dataset
import src.mace.mace as mace
import src.mace.loss as loss
import src.mace.train as train
import src.mace.test as test

Setup

In [3]:
# Read input file
now = datetime.datetime.now()
name = str(now.strftime("%Y%m%d")+'_'+now.strftime("%H%M%S"))
input_file = "./input/local.in"
path = f"./models/{name:s}"

# Set up PyTorch
cuda   = False
DEVICE = torch.device("cuda" if cuda else "cpu")
batch_size = 1 
kwargs = {'num_workers': 1, 'pin_memory': True} 

# Make output directories
utils.makeOutputDir(path)
utils.makeOutputDir(path+'/nn')

'./models/20241213_022215/nn'

In [4]:
## input model
n_dim       = 4
lr          = 1.e-5
nb_epochs   = 10
ini_epochs  = 5
losstype    = "abs_grd_idn"
z_dim       = 4
scheme      = "loc"
nb_samples  = 100
nb_evol     = 1
nb_hidden   = 1
ae_type     = "simple"
nb_test     = 5
dt_fract    = 1.0
p_dim       = 1

## loss fractions
abs = 1
grd = 1 
idn = 1
fract = {
    'abs' : abs, 
    'grd' : grd, 
    'idn' : idn,
    }

Load inputs

In [5]:
train_data, valid_data, train_loader, valid_loader = dataset.get_data(
    dt_fract=dt_fract,
    n_L96=n_dim,
    nb_samples=nb_samples,
    batch_size=batch_size, 
    nb_test=nb_test,
    kwargs=kwargs,
    )

Make model

In [6]:
model = mace.Solver(
    n_dim=n_dim,
    p_dim=p_dim,
    z_dim=z_dim,
    nb_hidden=nb_hidden,
    ae_type=ae_type,
    scheme=scheme,
    nb_evol=nb_evol,
    path=path,
    DEVICE=DEVICE,
    lr=lr,
    # g_nn=True,
    )
num_params = utils.count_parameters(model)
print(f'\nThe model has {num_params} trainable parameters')

4

The model has 3144 trainable parameters


Train - unnormalized losses

In [7]:
norm, fract = loss.initialise()

# Make loss objects
trainloss = loss.Loss(norm, fract, losstype)
testloss  = loss.Loss(norm, fract, losstype)

# Train
train.train(
    model, 
    train_loader,
    valid_loader, 
    end_epochs=ini_epochs, 
    trainloss=trainloss,
    testloss=testloss, 
    )


Local training scheme in use.

>>> Training model...


RuntimeError: shape '[4, 4]' is invalid for input of size 1599984

Train - normalized losses (reinitialize model)

In [ ]:
# Change the ratio of losses via the fraction
trainloss.change_fract(fract)
testloss.change_fract(fract)

# Normalise the losses
new_norm = trainloss.normalise()  
testloss.change_norm(new_norm) 

# Continue training
train.train(
    model, 
    train_loader,
    valid_loader, 
    start_epochs=ini_epochs,
    end_epochs=nb_epochs, 
    trainloss=trainloss,
    testloss=testloss, 
    )

Save model and data

In [ ]:
# losses
trainloss.save(path+'/train')
testloss.save(path+'/valid')

# model
torch.save(model.state_dict(),path+'/nn/nn.pt')

# status
np.save(path+'/train/status', model.get_status('train')) # type: ignore
np.save(path +'/valid/status', model.get_status('test') ) # type: ignore

fig_loss = loss.plot(trainloss, testloss, len = nb_epochs)
plt.show()

In [ ]:
loss.plot(
    trainloss,
    testloss,
    ylim=False,
    log=True,
    show=True,
    )

Test model on the test samples

In [ ]:
sum_err_step = 0
sum_err_evol = 0

step_calctime = list()
evol_calctime = list()

utils.makeOutputDir(path+'/figs')

for i in tqdm(range(len(train_data.testF))):
    testF = train_data.testF[i] 
    err_test, err_evol, step_time, evol_time,n, n_hat, n_evol  = test.test_model(
        model,
        n_dim,
        testF,
        printing=False,
        plotting=True,
        save=True,
        )
    sum_err_step += err_test
    sum_err_evol += err_evol

    step_calctime.append(step_time)
    evol_calctime.append(evol_time)

print('\nAverage error:')
print('           Step:', np.round(sum_err_step,3))
print('      Evolution:', np.round(sum_err_evol,3))
print('(following Eq. 23 of Maes et al., 2024)')